# Location Reachability

Seeing what coords are reachable within 1 hour of our work locations.

In [1]:
# map stuff
import googlemaps
import folium
from folium.plugins import HeatMap

import pandas as pd
import numpy as np
import json

In [2]:
with open('gmaps_key.json') as f:
    key = json.load(f)['key']

gmaps = googlemaps.Client(key = key)

## Functions

In [17]:
def get_distance(matrix_result):
    try:
        distance = matrix_result['rows'][0]['elements'][0]['distance']['text']
    except:
        distance = 'gmaps_failure'
    return distance

def get_time(matrix_result):
    try:
        time = matrix_result['rows'][0]['elements'][0]['duration']['text']
    except:
        time = 'gmaps_failure'
    return time

## Define Grid

In [23]:
# define grid lats and longs
lats = np.linspace(51.4, 51.6, 30)
longs = np.linspace(-.3, .1, 40)

coords = []

for lat in lats:
    for long in longs:
        coord = pd.DataFrame({'lat': [lat], 'long': [long]}) 
        coords.append(coord)

coords = pd.concat(coords)
map_data = [[row['lat'], row['long']] for index, row in coords.iterrows()]

m = folium.Map(location = [51.5, -0.1], zoom_start = 11)

# Plot it on the map
HeatMap(map_data, radius = 9).add_to(m)

# Display the map
m

### Check usage limits

Need to calculate travel times from 2 locations to all coordinates in grid. Therefore the number of call will be:

In [24]:
# number of coordinates
len(coords) * 2

2400

## Calculate travel times

In [25]:
def calc_travel_times(row, lat_work, long_work):
    lat = row['lat']
    long = row ['long']
    
    matrix_result = gmaps.distance_matrix(origins = (lat, long),
                                          destinations = (lat_work, long_work),
                                          mode = "transit")
    return matrix_result

In [31]:
gmaps_df = coords.copy()

gmaps_df['dan'] = gmaps_df.apply(lambda row: calc_travel_times(row, 51.5141, -0.2271), axis = 1)
gmaps_df['vi'] = gmaps_df.apply(lambda row: calc_travel_times(row, 51.5051, -0.0144), axis = 1)

Timeout: occurred at index 0

### Extract travel times and distances

In [ ]:
gmaps_df['travel_time_dan'] = gmaps_df['dan'].apply(get_time)
gmaps_df['travel_time_vi'] = gmaps_df['vi'].apply(get_time)

gmaps_df['distance_dan'] = gmaps_df['dan'].apply(get_distance)
gmaps_df['distance_vi'] = gmaps_df['vi'].apply(get_distance)

def time_to_minutes(time):
    if time == 'gmaps_failure':
        return 'gmaps_failure'
    time = time.split(' hour ')
    if len(time) > 1:
        hours = int(time[0])
        minutes = int(time[1][0:2])
        return int(hours * 60 + minutes)
    else:
        minutes = int(time[0][0:2])
        return int(minutes)
    
gmaps_df['mins_dan'] = gmaps_df['travel_time_dan'].apply(time_to_minutes)
gmaps_df['mins_vi'] = gmaps_df['travel_time_vi'].apply(time_to_minutes)

gmaps_df.head()

In [ ]:
gmaps_df.to_csv('gmaps_result.csv', index = False)

In [ ]:
reachable = gmaps_df[(gmaps_df['mins_dan'] != 'gmaps_failure') & (gmaps_df['mins_vi'] != 'gmaps_failure')].copy()

reachable['mins_dan'] = reachable['mins_dan'].apply(lambda x: int(x))
reachable['mins_vi'] = reachable['mins_vi'].apply(lambda x: int(x))

reachable = reachable[(reachable['mins_dan'] <= 60) & (reachable['mins_vi'] <= 60)]

map_data = [[row['lat'], row['long']] for index, row in reachable.iterrows()]

m = folium.Map(location = [51.5, -0.1], zoom_start = 11)

# Plot it on the map
HeatMap(map_data, radius = 9).add_to(m)

# Display the map
m